## CNN Basic
https://www.tensorflow.org/alpha/tutorials/images/intro_to_cnns

A CNN tutorial for classifying MNIST digits.

In [22]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

### Data preparation

In [23]:
(trainImages, trainLabels), (testImages, testLabels) = datasets.mnist.load_data()
print(trainImages.shape)
print(trainLabels.shape)
print(testImages.shape)
print(testLabels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Reshape and normalize the images.

In [24]:
trainImages = trainImages[...,None] / 255.
testImages = testImages[...,None] / 255.

The final dimensions

In [25]:
print(trainImages.shape)
print(testImages.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


correspond to `(batch, height, width, channels)`.

### Convolutional base
We will make a stack of `Conv2D` and `MaxPooling2D` layers. Our CNN will process inputs of shape `(28, 28, 1)`.

In [26]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=trainImages.shape[1:]),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The first few arguments of `Conv2D` and `MaxPool2D` are as follows:
- `Conv2D(filters, kernel_size[, strides][, padding])`
- `MaxPool2D([pool_size][, strides][, padding])`.

See how the height and width shrink and the number of channels increases as we go deeper in the network.

### Dense layers on top
To classify MNIST digits, we will feed the last output tensors of the convolutional base to one or more `Dense` layers. Because the CNN outputs are 3D tensors, we need to `Flatten` them before we feed them to `Dense`.

In [27]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

### Training

In [28]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(trainImages, trainLabels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 16s 268us/sample - loss: 0.1411 - accuracy: 0.9563
Epoch 2/5
60000/60000 [==============================] - 17s 277us/sample - loss: 0.0448 - accuracy: 0.9862
Epoch 3/5
60000/60000 [==============================] - 16s 271us/sample - loss: 0.0330 - accuracy: 0.9899
Epoch 4/5
60000/60000 [==============================] - 16s 269us/sample - loss: 0.0262 - accuracy: 0.9919
Epoch 5/5
60000/60000 [==============================] - 16s 266us/sample - loss: 0.0187 - accuracy: 0.9941


### Evaluation

In [30]:
testLoss, testAccuracy = model.evaluate(testImages, testLabels)
print(f"Test accuracy: {testAccuracy:.2%}")

10000/10000 [==============================] - 1s 66us/sample - loss: 0.0441 - accuracy: 0.9872
Test accuracy: 98.72%
